In [1]:
# List of candidates in the order listed on the voting site.
candidates = ["Muthiah Annamalai (PanMo Cloud)", "Alexandre Eichenberger (IBM)", "Mauro Bennici (Intelligenza Etica)", 
              "Andreas Fehlner (TRUMPF Laser AG)", "Ramakrishnan Sivakumar (AMD)", "Prasanth Pulavarthi (Microsoft)", 
              "Elvadas Nono (Oracle)", "Mayank Kaushik (NVIDIA)", "Aishik Ghosh (University of California Irvine)", 
              "Saurabh Tangri (Intel Corporation)", "Shraddha Goled (HCL Technologies)"]

# 4 letter identifier to enable simple identification in verbose results.
candidate_short = ["Muth", "Alex", "Maur", "Andr", "Rama", "Pras", "Elva", "Maya", "Aish", "Saur", "Shra"]

# Votes with company (added manually), 
#   name (replaced manually from email address, which should be avoided in published docs), 
#   and votes ranking for each candidate.
# Raw info can be pulled from the voting site as cvs file.
votes = [
["TRUMPF","Andreas Fehlner",11,2,8,1,4,3,10,6,7,5,9],
["IBM","Alexandre Eichenberger",11,1,11,1,6,1,11,1,11,1,11],
["MS","Ashay Rane",11,1,11,11,11,2,11,11,11,11,11],
["Tsavorite","Boyana Norris",8,1,6,4,11,2,11,9,3,9,7],
["QuantCo","Christian Bourjau",11,2,11,1,4,3,11,5,6,11,11],
["IBM","Charles Volzka",11,1,11,4,2,3,11,5,7,6,11],
["IBM","Michael Essenmacher",9,1,7,10,5,2,6,4,11,3,8],
["Intel","Freddy Chiu",10,2,9,5,4,3,8,6,7,1,11],
["MS","G Ramalingam",11,2,11,4,4,1,11,4,11,4,11],
["AMD","Daniel Holanda Noronha",11,4,11,11,1,4,11,4,5,4,11],
["IBM","Haruki Imai",11,1,11,11,11,2,11,2,11,11,11],
["Nuro","Chun-Wei (Jacky) Chen",11,2,11,2,2,1,11,11,11,2,11],
["NVIDIA","Joaquin Anton",3,1,3,1,2,1,3,1,3,2,3],
["Intel","Javier Martinez",8,2,7,6,10,4,11,5,3,1,9],
["AMD","Jeremy Fowers",11,11,11,2,1,2,11,11,11,2,11],
["MS","Justin Chu",9,2,9,2,2,1,5,2,9,2,9],
["QuantCo","Jakub Bachurski",7,1,10,2,8,6,11,4,5,3,9],
["NVIDIA","Kevin Chen",6,3,6,6,6,2,6,1,4,5,6],
["MS","Maanav Dalal",6,5,6,6,2,1,6,3,6,4,6],
["IBM","Yasushi Negishi",11,1,11,11,11,11,11,11,11,11,11],
["AMD","Philip Colangelo",11,11,11,11,1,11,11,11,11,11,11],
["Groq","Philip Lassen",11,1,11,3,1,5,11,6,11,4,11],
["MS","Prasanth Pulavarthi",10,2,3,2,2,1,3,2,3,2,10],
["MS","Stan Li",5,1,6,6,4,6,6,6,2,3,6],
["AMD","Ramakrishnan Sivakumar",11,2,11,2,1,2,11,3,11,2,11],
["GM","Ramakrishna Tipireddy",10,6,10,10,6,6,10,10,10,1,10],
["Amazon","Shrinivas Panchamuki Anand",11,11,11,11,1,3,11,11,11,2,11],
["OpenAI","Soren Lassen",10,1,11,9,3,2,6,5,7,4,8],
["Intel","Suyue Chen",6,6,5,4,5,6,2,5,3,1,5],
["Intel","Tai Huang",5,2,3,4,2,1,3,2,5,1,4],
["Preferred Networks","Takeshi Watanabe",7,9,10,11,4,2,8,5,1,3,6],
["IBM","Tabari Alexander",11,1,11,6,5,5,11,7,8,5,5],
["MS","Thiago Crepaldi",11,11,11,11,1,2,11,11,11,11,11],
["IBM","Tung Le",11,1,11,7,3,4,11,2,11,5,6],
["Riverbed","Venkat Vellaichamy",9,7,3,10,2,4,11,1,8,5,7],
["NVIDIA","Yuan Yao",11,2,11,4,11,11,11,1,3,5,11],
]

In [2]:
import numpy as np

# From the list of all ballots, seclect the ones for this company (or all if company is "ALL").
def ballot_for_company(ballots, select_company, verbose=0):
    res = []
    for b in ballots:
        curr_company, curr_ballot = b[0], b[2:]
        if select_company == 'ALL' or curr_company == select_company:
            res.append(curr_ballot)
            if verbose:
                print(curr_company, curr_ballot)
    return np.array(res)

# Help for debugging.
def justified_numbers(a):
    return map(lambda n: str(n).rjust(4), a)

def print_justified_numbers(title, a):
    print(title)
    print("  ", *candidate_short)
    print("  ", *justified_numbers(a))

# Create a new ballot reflecting the choices in the given ballot sample. 
# Verbose of 1 print debugging info, verbose of 2 additionally print every ballot used.
def rank(ballots, candidates, verbose=0):
    cnum = candidates.size
    num_ballots = 0
    #m[a,b] is "time folks preferred candidate b over a".
    m = np.zeros((cnum, cnum), dtype=int)
    for b in ballots:
        num_ballots += 1
        for j in range(cnum):
            for k in range(cnum):
                if int(b[k])>int(b[j]):
                    m[k][j] = m[k][j] + 1
    total = np.sum(m, axis=0) # sum over columns
    # (candidate id, preference score) ranked by most preferred first
    rank = sorted(zip(range(cnum),total), reverse=True, key=lambda record: record[1])
    rank_named = sorted(zip(candidate_short,total), reverse=True, key=lambda record: record[1])
    # New ballot: where candidates that have the same preferrence have the same score (low is better).
    new_ballot = np.zeros(cnum, dtype=int)
    preference = 0
    previous_score = 10000000
    for p in range(cnum): # By order of decreasing popularity.
        curr_candidate_id, curr_candidate_score = rank[p]
        if curr_candidate_score != previous_score:
            preference +=1 # Different score, advance by 1.
        new_ballot[curr_candidate_id] = preference
        previous_score = curr_candidate_score   
    if verbose:
        print("##########################################################")
        print("Tabulating", num_ballots, "votes.")
        if verbose > 1:
            i = 1
            for b in ballots:
                print("  ", i, ":", b)
                i += 1
        print("Matrix m[a, b], times voters preferred candidate b over a:")
        print("          ", *candidate_short, sep = " ")
        for i in range(cnum):
            m_i_str = justified_numbers(m[i])
            print("  ", i, candidate_short[i], ":", *m_i_str, sep = " ")
        print_justified_numbers("Totals (sum over columns of m)", total)
        print_justified_numbers("Rank (candidate id, candidate preference score)",rank_named)
        print_justified_numbers("New ballot", new_ballot)
    return (num_ballots, new_ballot)

# Create a ballot that reflects all the votes per company.
# Verbose of 1 print debugging info, verbose of 2 additionally print every ballot used.
def rank_per_company(votes, candidates, verbose=0):
    companies = np.unique(votes[:, 0])
    ballots_per_company = []
    ballots_per_company_with_name = []
    (tot_num_ballots, counted_num_ballot) = (np.shape(votes)[0], 0)
    for com in companies:
        print("Ballot for company = ", com)
        (company_num_ballots, new_ballot) = rank(ballot_for_company(votes, com), candidates, verbose=verbose)
        counted_num_ballot += company_num_ballots
        print("  is ", new_ballot, " with ", company_num_ballots, "counted ballots\n")
        ballots_per_company.append(new_ballot)
        ballots_per_company_with_name.append([com, new_ballot])
    assert counted_num_ballot == tot_num_ballots
    print("Final ballot with one vote per company")
    (final_num_ballot, final_ballot) = rank(np.array(ballots_per_company), candidates, verbose=verbose)
    print("  is ", final_ballot, " with ", final_num_ballot, "counted ballots\n")
    assert final_num_ballot == np.shape(companies)[0]
    if verbose:
        print("Final ballot = ", final_ballot)
        print("")
    return (counted_num_ballot, final_ballot, ballots_per_company_with_name)

# Translate a ballot into the list of ordered candidates using names.
def ballot_to_candidates(ballot, candidates):
    cnum = candidates.size
    # (candidate id, order) ranked by most preferred first
    rank = sorted(zip(range(cnum), ballot), key=lambda record: record[1])
    for r in rank:
        num = str(r[1])+". "
        print(num, candidates[r[0]])

In [3]:
# Use numpy arrays.
vv = np.array(votes)
cc = np.array(candidates)
verbose = 1 # 0 none, 1 some, 2 print ballots also.
#ballot_for_company(vv, 'INTEL')
#rank(ballot_for_company(vv, "INTEL"), cc, verbose=1)

print("Non-official vote where each voter get one voice. This is not the offical result and is only provided to investgate the difference between individual votes vs votes per company\n")
(total_votes1, ballot_per_voter) = rank(ballot_for_company(vv, "ALL"), cc, verbose=verbose)
print("\nBallot per voter =", ballot_per_voter)
ballot_to_candidates(ballot_per_voter, cc)
print("\n\n")

print("Official vote per our election rules, where votes are tallied by company first, and the final round includes exactly one ballot per company\n")
(total_votes2, final_ballot_per_company, ballots_per_company) = rank_per_company(vv, cc, verbose=verbose)
assert total_votes1 == total_votes2
print("\nFinal ballot (summarizing per company)=", final_ballot_per_company)
ballot_to_candidates(final_ballot_per_company, cc)



Non-official vote where each voter get one voice. This is not the offical result and is only provided to investgate the difference between individual votes vs votes per company

##########################################################
Tabulating 36 votes.
Matrix m[a, b], times voters preferred candidate b over a:
           Muth Alex Maur Andr Rama Pras Elva Maya Aish Saur Shra
   0 Muth :    0   30    9   22   28   31    8   26   17   29   10
   1 Alex :    1    0    2    4   10   12    2    6    2   10    2
   2 Maur :    4   30    0   21   28   30    6   25   15   28    5
   3 Andr :    4   23    4    0   19   23    5   12   10   16    6
   4 Rama :    3   20    2    9    0   20    1   11    8   13    2
   5 Pras :    1   17    2    6   12    0    1    6    6    9    1
   6 Elva :    6   30    5   20   29   31    0   26   14   30    8
   7 Maya :    2   20    1   12   18   23    1    0    5   17    2
   8 Aish :    1   30    3   18   25   28    5   23    0   25    5
   9 Saur :   

In [4]:
def printResultMD(year, ballots, aggregated_ballots, candidates, candidate_shortname, results, verbose=0):
    cnum = candidates.size
    # Table of votes
    print("# Vote results for year " + year + ".")
    print("## Votes casted by Contributors.")
    str = "| Contributors | "
    for c in candidates:
        str += c + " | "
    print(str)
    str = "|"
    for c in range(cnum+1):
        str += "-------|"
    print(str)
    for b in ballots:
        str = "| " + b[1] + " (" + b[0] + ") | "
        for c in range(cnum):
            str += b[c+2] + " | "
        print(str)
    # Print ballots per company
    print("\n## Precedence per company.")
    print("| Member company | Perferences |")
    print("|----------------|-------------------------------------------------------------|")
    for b in aggregated_ballots:
        bb = np.array(b[1])
        rank = sorted(zip(candidate_shortname,bb), key=lambda record: record[1])
        if (verbose):
            print("ballot:", b)
            print("rank ", rank)
        str = "| " + b[0] + " | "
        for c in range(cnum):
            if c>0:
                if rank[c][1] == rank[c-1][1]:
                    str += " = "
                else:
                    str += " > "
            str += rank[c][0]
        print(str+ " |")        
    # Print final results
    print("\n## Final results (Schultz method, top 5 are elected).")
    ballot_to_candidates(results, candidates)

In [5]:
# Copy output in a separate file for easy consumption of the results.
printResultMD("2024-2025", vv, ballots_per_company, cc, candidate_short, final_ballot_per_company)

# Vote results for year 2024-2025.
## Votes casted by Contributors.
| Contributors | Muthiah Annamalai (PanMo Cloud) | Alexandre Eichenberger (IBM) | Mauro Bennici (Intelligenza Etica) | Andreas Fehlner (TRUMPF Laser AG) | Ramakrishnan Sivakumar (AMD) | Prasanth Pulavarthi (Microsoft) | Elvadas Nono (Oracle) | Mayank Kaushik (NVIDIA) | Aishik Ghosh (University of California Irvine) | Saurabh Tangri (Intel Corporation) | Shraddha Goled (HCL Technologies) | 
|-------|-------|-------|-------|-------|-------|-------|-------|-------|-------|-------|-------|
| Andreas Fehlner (TRUMPF) | 11 | 2 | 8 | 1 | 4 | 3 | 10 | 6 | 7 | 5 | 9 | 
| Alexandre Eichenberger (IBM) | 11 | 1 | 11 | 1 | 6 | 1 | 11 | 1 | 11 | 1 | 11 | 
| Ashay Rane (MS) | 11 | 1 | 11 | 11 | 11 | 2 | 11 | 11 | 11 | 11 | 11 | 
| Boyana Norris (Tsavorite) | 8 | 1 | 6 | 4 | 11 | 2 | 11 | 9 | 3 | 9 | 7 | 
| Christian Bourjau (QuantCo) | 11 | 2 | 11 | 1 | 4 | 3 | 11 | 5 | 6 | 11 | 11 | 
| Charles Volzka (IBM) | 11 | 1 | 11 | 4 | 2 | 3 